<a href="https://colab.research.google.com/github/once-upon-an-april/Thuc-Hanh-Deep-Learning-trong-Khoa-Hoc-Du-Lieu-DS201.Q11.1/blob/main/Bai2/LAB_3_Demo_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN đơn giản (one-to-one, many-to-one)

In [ ]:
import tensorflow as tf

# Cấu hình mô hình RNN nhiều lớp (many-to-one)
num_classes = 5  # ví dụ 5 lớp phân loại
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=32),   # Embedding từ từ thành vector
    tf.keras.layers.SimpleRNN(64),                              # RNN layer với 64 đơn vị ẩn
    tf.keras.layers.Dense(num_classes, activation='softmax')    # Lớp output với softmax
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# LSTM

In [ ]:
import tensorflow as tf

# Ví dụ LSTM cho phân loại chuỗi (many-to-one)
num_classes = 3  # ví dụ 3 lớp phân loại
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64),   # Embedding với 64 chiều
    tf.keras.layers.LSTM(128),                                  # LSTM layer với 128 đơn vị ẩn
    tf.keras.layers.Dense(num_classes, activation='softmax')    # Output softmax
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# GRU

In [ ]:
import tensorflow as tf

# Ví dụ GRU cho phân loại chuỗi
num_classes = 4
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64),   # Embedding
    tf.keras.layers.GRU(64),                                    # GRU layer với 64 đơn vị ẩn
    tf.keras.layers.Dense(num_classes, activation='softmax')    # Output softmax
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# BiLSTM

In [ ]:
import tensorflow as tf

# Ví dụ BiLSTM cho phân loại chuỗi (many-to-one)
num_classes = 2
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64),     # Embedding
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),      # BiLSTM với 64 đơn vị (32 mỗi chiều)
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Mạng LSTM xếp chồng

In [ ]:
import tensorflow as tf

# Ví dụ Stacked LSTM (2 tầng)
num_classes = 5
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64),
    tf.keras.layers.LSTM(128, return_sequences=True),  # LSTM tầng 1 (xuất ra chuỗi)
    tf.keras.layers.LSTM(128),                         # LSTM tầng 2 (xuất ra ẩn cuối)
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Pipeline xử lý văn bản (Phân loại)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Dữ liệu văn bản mẫu
texts = [
    "Tôi yêu học máy",
    "Mạng nơ-ron rất mạnh mẽ",
    "Học sâu trong NLP"
]
labels = [1, 0, 1]  # Ví dụ nhãn nhị phân

# 2. Tokenization & tạo từ điển
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# 3. Đệm chuỗi về độ dài bằng nhau
data = pad_sequences(sequences, maxlen=5)
print(data)  # Mỗi câu được chuyển thành mảng số nguyên cố định độ dài

# Nhận dạng Thực thể có tên (NER) với BiLSTM

In [ ]:
import tensorflow as tf

# Giả sử có 3 loại nhãn (ví dụ: PERSON, ORG, O)
num_tags = 3
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    ),
    tf.keras.layers.TimeDistributed(
        tf.keras.layers.Dense(num_tags, activation='softmax')
    )
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# EDA DATASET

Yêu cầu: Thực hiện EDA và tiền xử lý 2 bộ dataset trước khi làm bài lab

## BỘ DỮ LIỆU UIT-VSFC (Phân loại văn bản)

1. Kiểm tra cấu trúc bộ dữ liệu

* Tải và đọc các file .txt hoặc .csv từ UIT-VSFC.

* Kiểm tra các cột có trong dataset (ví dụ: sentence, label).

* Kiểm tra số lượng mẫu train/valid/test.

* Lấy 10 mẫu đầu tiên để xem dữ liệu thực trông như thế nào.

In [ ]:
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
# Tải bộ dữ liệu UIT-VSFC
# 1. Tải dataset
print(">>> Đang tải dataset (phiên bản Parquet)...")
try:
    ds = load_dataset(
        "uitnlp/vietnamese_students_feedback",
        name="default", # Cấu hình mặc định
        revision="refs/convert/parquet" # Chỉ định nhánh chứa file parquet
    )
except Exception as e:
    print("Lỗi khi tải trực tiếp, chuyển sang phương án tải thủ công (bên dưới).")
    raise e

# Kiểm tra các cột có trong dataset
print("="*30)
print("CẤU TRÚC DATASET")
print("="*30)
print(f"Các cột trong tập Train: {ds['train'].column_names}")
print("Chi tiết Features:")
print(ds['train'].features)

# 3. Kiểm tra số lượng mẫu
print("\n" + "="*30)
print("SỐ LƯỢNG MẪU")
print("="*30)
print(f"- Train set:      {len(ds['train'])} mẫu")
print(f"- Validation set: {len(ds['validation'])} mẫu")
print(f"- Test set:       {len(ds['test'])} mẫu")
print(f"-> Tổng cộng:     {len(ds['train']) + len(ds['validation']) + len(ds['test'])} mẫu")

# 4. Xem 10 mẫu dữ liệu thực tế đầu tiên
print("\n" + "="*30)
print("10 MẪU DỮ LIỆU ĐẦU TIÊN (Tập Train)")
print("="*30)

2. Phân tích phân bố nhãn (Label Distribution)

* Thống kê tần suất mỗi nhãn (Positive / Negative / Neutral).

* Vẽ biểu đồ (bar chart) phân bố nhãn.

* **Trả lời câu hỏi**: Bộ dữ liệu có bị mất cân bằng nhãn không?

In [ ]:
# 1. Chuyển tập TRAIN sang DataFrame để dễ tính toán
df_train = pd.DataFrame(ds['train'])

# 2. Thống kê tần suất mỗi nhãn
# Map từ số sang chữ cho dễ hiểu: 0: Negative, 1: Neutral, 2: Positive
label_map = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
df_train['label_name'] = df_train['sentiment'].map(label_map)

# Đếm số lượng và sắp xếp theo thứ tự logic: Tiêu cực -> Trung tính -> Tích cực
counts = df_train['label_name'].value_counts()
desired_order = ['Negative', 'Neutral', 'Positive']
counts = counts.reindex(desired_order)

print("\n" + "="*30)
print("THỐNG KÊ PHÂN BỐ NHÃN (Tập Train)")
print("="*30)
print(counts)

# Tính tỷ lệ phần trăm
percentages = (counts / len(df_train)) * 100
print("\nTỷ lệ phần trăm:")
print(percentages.map('{:.2f}%'.format))

# 3. Vẽ biểu đồ Bar Chart
plt.figure(figsize=(8, 5))
colors = ['#d62728', '#7f7f7f', '#2ca02c'] # Đỏ (Neg), Xám (Neu), Xanh (Pos)
bars = plt.bar(counts.index, counts.values, color=colors)

plt.title('Phân bố nhãn cảm xúc (Sentiment) - Tập Train')
plt.xlabel('Nhãn (Label)')
plt.ylabel('Số lượng mẫu')
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Hiển thị con số cụ thể trên đầu mỗi cột
plt.bar_label(bars, fmt='%d')
plt.show()

# 4. Trả lời câu hỏi: Có mất cân bằng không?
print("\n" + "="*30)
print("KẾT LUẬN VỀ MẤT CÂN BẰNG")
print("="*30)
neutral_percent = percentages['Neutral']
if neutral_percent < 15: # Ngưỡng thường dùng để xác định mất cân bằng nặng
    print(f"-> CÓ. Bộ dữ liệu bị mất cân bằng nặng.")
    print(f"-> Lý do: Nhãn 'Neutral' chỉ chiếm {neutral_percent:.2f}%, thấp hơn rất nhiều so với Negative và Positive.")
    print("-> Khuyến nghị: Cần dùng kỹ thuật Oversampling, Class Weighting hoặc Stratified Split khi train.")
else:
    print("-> Bộ dữ liệu tương đối cân bằng.")

3. Phân tích độ dài câu

* Tính số token/word trong từng câu.

* Vẽ histogram hoặc boxplot độ dài câu.

Tìm: Max sequence length hợp lý khi padding (ví dụ: 50, 100, 200).

In [ ]:
import numpy as np
import seaborn as sns

# --- PHẦN 3: PHÂN TÍCH ĐỘ DÀI CÂU ---

# 1. Tính số từ (tokens) trong mỗi câu
# Lưu ý: Ở đây tách theo khoảng trắng (space) để ước lượng nhanh.
# Nếu làm kỹ hơn cho tiếng Việt, bạn có thể dùng pyvi hoặc VnCoreNLP.
df_train['num_tokens'] = df_train['sentence'].astype(str).apply(lambda x: len(x.split()))

# 2. Thống kê cơ bản
print("\n" + "="*30)
print("THỐNG KÊ ĐỘ DÀI CÂU")
print("="*30)
print(df_train['num_tokens'].describe())

# 3. Vẽ biểu đồ (Histogram & Boxplot)
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# Histogram: Xem phân phối chung
sns.histplot(df_train['num_tokens'], bins=30, kde=True, color='skyblue', ax=ax[0])
ax[0].set_title('Phân phối độ dài câu (Histogram)')
ax[0].set_xlabel('Số lượng từ')
ax[0].set_ylabel('Số lượng câu')

# Boxplot: Xem outlier (câu quá dài)
sns.boxplot(x=df_train['num_tokens'], color='lightgreen', ax=ax[1])
ax[1].set_title('Biểu đồ hộp (Boxplot) xác định Outliers')
ax[1].set_xlabel('Số lượng từ')

plt.tight_layout()
plt.show()

# 4. Tìm Max Sequence Length hợp lý
print("\n" + "="*30)
print("CHỌN MAX SEQUENCE LENGTH (PADDING)")
print("="*30)

# Tính các mốc phần trăm (Quantiles)
quantile_90 = df_train['num_tokens'].quantile(0.90)
quantile_95 = df_train['num_tokens'].quantile(0.95)
quantile_99 = df_train['num_tokens'].quantile(0.99)
max_len_real = df_train['num_tokens'].max()

print(f"- Độ dài bao phủ 90% dữ liệu: {int(quantile_90)} từ")
print(f"- Độ dài bao phủ 95% dữ liệu: {int(quantile_95)} từ")
print(f"- Độ dài bao phủ 99% dữ liệu: {int(quantile_99)} từ")
print(f"- Câu dài nhất thực tế:       {max_len_real} từ")

# Đề xuất
suggested_len = int(quantile_95)
print(f"\n-> KHUYẾN NGHỊ: Nên chọn max_len khoảng {suggested_len} - {suggested_len + 5}.")
print(f"   (Lý do: Bao phủ được 95% dữ liệu, tránh lãng phí bộ nhớ do padding quá nhiều cho các câu outlier dài {max_len_real} từ).")

4. Thống kê từ vựng

* Đếm số lượng từ vựng (unique tokens).

* Top 20 từ xuất hiện nhiều nhất.

* Trả lời câu hỏi: Có cần làm preprocessing không? (lowercase, remove punctuation, fix unicode…)

In [ ]:
from collections import Counter
import pandas as pd

# --- PHẦN 4: THỐNG KÊ TỪ VỰNG (VOCABULARY) ---

# 1. Tạo danh sách tất cả các token (tách đơn giản bằng khoảng trắng để kiểm tra thô)
# Lưu ý: Đây chưa phải là word segmentation chuẩn tiếng Việt, chỉ để kiểm tra độ sạch của dữ liệu.
all_words = " ".join(df_train['sentence'].astype(str)).split()

# 2. Đếm số lượng từ vựng (Unique tokens)
word_counts = Counter(all_words)
vocab_size = len(word_counts)

print("\n" + "="*30)
print("THỐNG KÊ TỪ VỰNG (Raw Data)")
print("="*30)
print(f"Tổng số từ (Total tokens):   {len(all_words)}")
print(f"Kích thước từ điển (Vocab):  {vocab_size} từ (unique)")

# 3. Top 20 từ xuất hiện nhiều nhất
print("\n" + "="*30)
print("TOP 20 TỪ XUẤT HIỆN NHIỀU NHẤT")
print("="*30)
top_20 = word_counts.most_common(20)

# Hiển thị dạng bảng cho dễ nhìn
df_top20 = pd.DataFrame(top_20, columns=['Từ (Token)', 'Số lần xuất hiện'])
print(df_top20)

# 4. Kiểm tra nhanh một số trường hợp cụ thể để quyết định preprocessing
print("\n" + "="*30)
print("KIỂM TRA NHU CẦU PREPROCESSING")
print("="*30)
print(f"- 'thầy' xuất hiện: {word_counts['thầy']} lần")
print(f"- 'Thầy' xuất hiện: {word_counts['Thầy']} lần (Case sensitive?)")
print(f"- 'tốt'  xuất hiện: {word_counts['tốt']} lần")
print(f"- 'tốt.' xuất hiện: {word_counts['tốt.']} lần (Dính dấu câu?)")
print(f"- 'ko'   xuất hiện: {word_counts['ko']} lần (Teencode?)")

5. Tiền xử lý dữ liệu

* Sinh viên phải mô tả:

* Các bước preprocessing sẽ áp dụng cho text: chuẩn hóa unicode, tách từ (tokenization dùng VnCoreNLP / underthesea / pyvi…), lowercasing, xử lý emoji, loại bỏ ký tự thừa, ...

Minh hoạ bằng ví dụ before → after.

In [ ]:
!pip install pyvi

import re
from pyvi import ViTokenizer
import string

# Hàm tiền xử lý toàn diện
def preprocess_text(text):
    # 1. Chuyển về chữ thường (Lowercase)
    text = text.lower()

    # 2. Loại bỏ các ký tự đặc biệt, dấu câu không cần thiết
    # Giữ lại các từ tiếng Việt và số, loại bỏ emoji hoặc ký tự lạ
    # Cách đơn giản: Xóa các ký tự trong chuỗi string.punctuation
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)

    # 3. Tách từ tiếng Việt (Word Segmentation) -> QUAN TRỌNG NHẤT
    # Ví dụ: "sinh viên" -> "sinh_viên"
    text = ViTokenizer.tokenize(text)

    return text

# --- ÁP DỤNG VÀO DATAFRAME ---
print(">>> Đang thực hiện Preprocessing...")

# Tạo cột mới 'sentence_cleaned' để giữ lại dữ liệu gốc đối chiếu
df_train['sentence_cleaned'] = df_train['sentence'].astype(str).apply(preprocess_text)

# Xem kết quả so sánh
print("\n" + "="*30)
print("SO SÁNH TRƯỚC VÀ SAU KHI XỬ LÝ")
print("="*30)
print(df_train[['sentence', 'sentence_cleaned']].head(5))

# Kiểm tra lại Top từ vựng sau khi xử lý
print("\n" + "="*30)
print("TOP 10 TỪ VỰNG SAU KHI TÁCH TỪ (Preprocessed)")
print("="*30)
all_words_clean = " ".join(df_train['sentence_cleaned']).split()
print(pd.Series(all_words_clean).value_counts().head(10))

In [ ]:
# Chuyển đổi Valid và Test sang DataFrame
df_valid = pd.DataFrame(ds['validation'])
df_test = pd.DataFrame(ds['test'])

print(">>> Đang xử lý tập Validation và Test...")
# Áp dụng hàm preprocess_text đã viết ở bước trước
df_valid['sentence_cleaned'] = df_valid['sentence'].astype(str).apply(preprocess_text)
df_test['sentence_cleaned'] = df_test['sentence'].astype(str).apply(preprocess_text)

print("Đã xử lý xong toàn bộ dữ liệu!")

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Gom toàn bộ văn bản sạch lại thành 1 chuỗi dài
all_text = " ".join(df_train['sentence_cleaned'])

# Tạo WordCloud
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    colormap='viridis',
    max_words=100
).generate(all_text)

# Vẽ biểu đồ
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('WordCloud - Các từ phổ biến nhất trong UIT-VSFC')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

print("\n" + "="*30)
print("VECTOR HÓA DỮ LIỆU (TF-IDF)")
print("="*30)

# Cấu hình TF-IDF
# max_features=5000: Chỉ giữ lại 5000 từ quan trọng nhất để giảm nhiễu và nhẹ máy
# ngram_range=(1, 1): Chỉ lấy từ đơn (vì đã ghép từ bằng pyvi rồi nên không cần ngram 2-3 nữa)
tfidf = TfidfVectorizer(max_features=5000)

# Học từ vựng từ tập Train và biến đổi dữ liệu
X_train_tfidf = tfidf.fit_transform(df_train['sentence_cleaned'])

# Chỉ biến đổi tập Valid/Test (KHÔNG fit lại để tránh data leakage)
X_valid_tfidf = tfidf.transform(df_valid['sentence_cleaned'])
X_test_tfidf = tfidf.transform(df_test['sentence_cleaned'])

# Lấy nhãn (y)
y_train = df_train['sentiment']
y_valid = df_valid['sentiment']
y_test = df_test['sentiment']

print(f"Kích thước tập Train sau khi vector hóa: {X_train_tfidf.shape}")
print(f"Kích thước tập Test sau khi vector hóa:  {X_test_tfidf.shape}")
print("-" * 30)
print("Ví dụ: Từ vựng index 100 đến 110 trong từ điển:")
print(list(tfidf.vocabulary_.keys())[100:110])

## BỘ DỮ LIỆU PhoNERT

1. Kiểm tra cấu trúc dữ liệu

* Tải và đọc các file train, test, dev.

* In 20 dòng đầu để xem cấu trúc dữ liệu BIO

In [ ]:
import json
import pandas as pd
import requests

# 1. Tải dữ liệu từ GitHub VinAIResearch
urls = {
    "train": "https://raw.githubusercontent.com/VinAIResearch/PhoNER_COVID19/master/data/word/train_word.json",
    "dev": "https://raw.githubusercontent.com/VinAIResearch/PhoNER_COVID19/master/data/word/dev_word.json",
    "test": "https://raw.githubusercontent.com/VinAIResearch/PhoNER_COVID19/master/data/word/test_word.json"
}

print(">>> Đang tải dữ liệu từ GitHub VinAIResearch (JSONL Format)...")
data = {}

for split, url in urls.items():
    print(f"Downloading {split}...")
    response = requests.get(url)
    if response.status_code == 200:
        # --- SỬA LỖI TẠI ĐÂY ---
        # File dạng JSONL: Tách từng dòng ra và load json cho từng dòng
        lines = response.text.strip().split('\n')
        data[split] = [json.loads(line) for line in lines]
    else:
        print(f"Lỗi tải file {split}: {response.status_code}")

# 2. Kiểm tra cấu trúc chung
print("\n" + "="*30)
print("CẤU TRÚC BỘ DỮ LIỆU")
print("="*30)
sample_structure = data['train'][0]
print(f"Keys trong 1 mẫu: {list(sample_structure.keys())}")
# Mong đợi: ['words', 'tags']

# 3. Kiểm tra số lượng mẫu
print("\n" + "="*30)
print("SỐ LƯỢNG MẪU (Câu)")
print("="*30)
print(f"Train set: {len(data['train'])} câu")
print(f"Dev set:   {len(data['dev'])} câu")
print(f"Test set:  {len(data['test'])} câu")

# 4. In 20 dòng đầu (Token - Tag) của mẫu đầu tiên
print("\n" + "="*30)
print("XEM CHI TIẾT CẤU TRÚC BIO (Câu đầu tiên tập Train)")
print("="*30)

first_sample = data['train'][0]
df_sample = pd.DataFrame({
    'Word': first_sample['words'],
    'Tag': first_sample['tags']
})

print(df_sample.head(20))

# 5. Xem nhanh vài câu khác
print("\n" + "="*30)
print("XEM NHANH 5 CÂU KHÁC")
print("="*30)
for i in range(1, 6):
    sample = data['train'][i]
    # Ghép từ và tag lại xem cho gọn
    # Chỉ in 100 ký tự đầu
    text_preview = " ".join([f"{w}/{t}" for w, t in zip(sample['words'], sample['tags'])])
    print(f"Câu {i}: {text_preview[:100]}...")

2. Thống kê số câu trong từng tập

* Số câu trong train/dev/test

* Số lượng token mỗi tập

In [ ]:
import pandas as pd

# Giả sử biến 'data' đã được tải thành công từ bước trước
# data = {'train': [...], 'dev': [...], 'test': [...]}

print(">>> Đang tính toán thống kê...")

stats = []

for split in ['train', 'dev', 'test']:
    # 1. Lấy danh sách các câu trong tập hiện tại
    samples = data[split]

    # 2. Đếm số câu
    num_sentences = len(samples)

    # 3. Đếm tổng số token
    # Duyệt qua từng câu, lấy độ dài của list 'words' và cộng dồn
    num_tokens = sum(len(s['words']) for s in samples)

    # 4. Tính độ dài trung bình (để biết câu dài hay ngắn)
    avg_len = num_tokens / num_sentences if num_sentences > 0 else 0

    # Lưu vào danh sách kết quả
    stats.append({
        'Tập dữ liệu': split.upper(),
        'Số câu': num_sentences,
        'Tổng số token': num_tokens,
        'Trung bình (Token/Câu)': round(avg_len, 2)
    })

# Chuyển thành DataFrame hiển thị cho đẹp
df_stats = pd.DataFrame(stats)

print("\n" + "="*30)
print("BẢNG THỐNG KÊ CHI TIẾT")
print("="*30)
print(df_stats)

# Vẽ biểu đồ so sánh số lượng câu (Optional)
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
bars = plt.bar(df_stats['Tập dữ liệu'], df_stats['Số câu'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.title('Số lượng câu trong các tập dữ liệu (PhoNER_COVID19)')
plt.ylabel('Số câu')
plt.bar_label(bars, fmt='%d')
plt.show()

3. Phân bố nhãn thực thể

* Thống kê số lần xuất hiện của từng nhãn (B-PER, I-PER, B-LOC, I-LOC, B-ORG, I-ORG, O,...).

* Vẽ biểu đồ bar chart phân bố nhãn.

Trả lời:

* Nhãn nào xuất hiện nhiều nhất?

* Nhãn nào xuất hiện hiếm?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pandas as pd

# --- PHẦN 3: PHÂN TÍCH PHÂN BỐ NHÃN (LABEL DISTRIBUTION) ---

# Giả sử biến 'data' đã có từ code trước (data['train'], data['dev'], data['test'])
print(">>> Đang phân tích nhãn trong tập TRAIN...")

# 1. Gom tất cả các nhãn trong tập Train lại thành 1 list duy nhất
all_tags = []
for sample in data['train']:
    all_tags.extend(sample['tags'])

# 2. Đếm số lượng mỗi nhãn
tag_counts = Counter(all_tags)

# Chuyển sang DataFrame để dễ quan sát và sắp xếp
df_tags = pd.DataFrame.from_dict(tag_counts, orient='index', columns=['Count']).reset_index()
df_tags.columns = ['Label', 'Count']

# Tính tỷ lệ phần trăm
total_tags = len(all_tags)
df_tags['Percentage'] = (df_tags['Count'] / total_tags) * 100

# Sắp xếp giảm dần
df_tags = df_tags.sort_values(by='Count', ascending=False)

print("\n" + "="*30)
print("BẢNG THỐNG KÊ TẦN SUẤT NHÃN")
print("="*30)
print(df_tags)

# 3. Vẽ biểu đồ (Bar Chart)
# Lưu ý: Nhãn 'O' thường chiếm áp đảo (90%), nên ta sẽ vẽ 2 biểu đồ để nhìn rõ hơn.

fig, ax = plt.subplots(1, 2, figsize=(18, 6))

# Biểu đồ 1: Toàn bộ nhãn (Bao gồm 'O')
sns.barplot(data=df_tags, x='Label', y='Count', ax=ax[0], palette='viridis')
ax[0].set_title('Phân bố toàn bộ nhãn (Bao gồm O)')
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
ax[0].set_ylabel('Số lượng')

# Biểu đồ 2: Chỉ các nhãn thực thể (Loại bỏ 'O')
df_entities = df_tags[df_tags['Label'] != 'O']
sns.barplot(data=df_entities, x='Label', y='Count', ax=ax[1], palette='magma')
ax[1].set_title('Phân bố các nhãn Thực thể (Bỏ qua O)')
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
ax[1].set_ylabel('Số lượng')

plt.tight_layout()
plt.show()

# 4. Trả lời câu hỏi
print("\n" + "="*30)
print("KẾT LUẬN")
print("="*30)

# Nhãn nhiều nhất
top_1 = df_tags.iloc[0]
# Nhãn thực thể nhiều nhất (không tính O)
top_entity = df_entities.iloc[0]
# Nhãn hiếm nhất
min_1 = df_tags.iloc[-1]

print(f"1. Nhãn xuất hiện nhiều nhất toàn cục: '{top_1['Label']}' ({top_1['Count']} lần - chiếm {top_1['Percentage']:.2f}%)")
print(f"   -> Đây là nhãn 'Outside' (không phải thực thể).")
print(f"\n2. Nhãn THỰC THỂ xuất hiện nhiều nhất: '{top_entity['Label']}' ({top_entity['Count']} lần)")
print(f"   -> Điều này phản ánh dataset tập trung vào thông tin này (thường là LOCATION hoặc PATIENT_ID).")
print(f"\n3. Nhãn xuất hiện HIẾM nhất: '{min_1['Label']}' ({min_1['Count']} lần)")
print(f"   -> Cần lưu ý khi train, model có thể học kém ở nhãn này (Class Imbalance).")

4. Phân tích độ dài câu

* Tính số token cho từng câu.

* Tìm min / max / mean.

Trả lời câu hỏi: padding tối ưu? có câu nào quá dài gây bất lợi cho LSTM?

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# --- PHẦN 4: PHÂN TÍCH ĐỘ DÀI CÂU (SEQUENCE LENGTH) ---

print(">>> Đang phân tích độ dài câu trong tập TRAIN...")

# 1. Tính số token cho từng câu
# List chứa độ dài của từng câu
sent_lengths = [len(sample['words']) for sample in data['train']]

# Chuyển sang DataFrame để thống kê nhanh
df_len = pd.DataFrame(sent_lengths, columns=['length'])

# 2. Tìm Min / Max / Mean / Median
print("\n" + "="*30)
print("THỐNG KÊ ĐỘ DÀI CÂU")
print("="*30)
stats = df_len.describe()
print(stats)

min_len = df_len['length'].min()
max_len = df_len['length'].max()
mean_len = df_len['length'].mean()
median_len = df_len['length'].median()

print(f"\n- Ngắn nhất (Min): {min_len} tokens")
print(f"- Dài nhất (Max):  {max_len} tokens")
print(f"- Trung bình (Mean): {mean_len:.2f} tokens")
print(f"- Trung vị (Median): {median_len} tokens")

# 3. Tính các mốc Percentile (Quan trọng cho việc chọn Padding)
p90 = np.percentile(sent_lengths, 90)
p95 = np.percentile(sent_lengths, 95)
p99 = np.percentile(sent_lengths, 99)

print("\n" + "="*30)
print("CÁC MỐC PHẦN TRĂM (PERCENTILES)")
print("="*30)
print(f"- 90% số câu có độ dài <= {p90} tokens")
print(f"- 95% số câu có độ dài <= {p95} tokens")
print(f"- 99% số câu có độ dài <= {p99} tokens")

# 4. Vẽ biểu đồ Histogram và Boxplot
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# Histogram: Xem phân phối chuẩn hay lệch
sns.histplot(sent_lengths, bins=30, kde=True, ax=ax[0], color='skyblue')
ax[0].set_title('Phân phối độ dài câu (Histogram)')
ax[0].set_xlabel('Số lượng token')
ax[0].set_ylabel('Số lượng câu')
ax[0].axvline(p95, color='r', linestyle='--', label=f'95% ({int(p95)})')
ax[0].legend()

# Boxplot: Phát hiện câu quá dài (Outliers)
sns.boxplot(x=sent_lengths, ax=ax[1], color='lightgreen')
ax[1].set_title('Biểu đồ hộp (Boxplot) phát hiện Outlier')
ax[1].set_xlabel('Số lượng token')

plt.tight_layout()
plt.show()

# --- TRẢ LỜI CÂU HỎI ---
print("\n" + "="*30)
print("TRẢ LỜI CÂU HỎI")
print("="*30)

# Câu 1: Padding tối ưu?
# Thường chọn mốc bao phủ 95% đến 99% dữ liệu, và làm tròn lên lũy thừa của 2 (32, 64, 128...)
optimal_padding = int(p99)
if optimal_padding < 64: suggested_maxlen = 64
elif optimal_padding < 128: suggested_maxlen = 128
elif optimal_padding < 256: suggested_maxlen = 256
else: suggested_maxlen = 512

print(f"1. Padding tối ưu (max_len):")
print(f"   - Nếu chọn theo 99% dữ liệu: {int(p99)} tokens.")
print(f"   - Khuyến nghị cấu hình Model: max_len = {suggested_maxlen}.")
print(f"   (Lý do: Bao phủ gần hết dữ liệu mà không tốn tài nguyên cho các câu outlier quá dài).")

# Câu 2: Có câu nào quá dài gây bất lợi cho LSTM?
outlier_threshold = p99 + 20 # Giả sử dài hơn mốc 99% một đoạn là bất lợi
long_sentences = df_len[df_len['length'] > 100].count().values[0] # LSTM thường bắt đầu khó khăn > 100 bước
print(f"\n2. Có câu nào quá dài gây bất lợi cho LSTM không?")
if max_len > 100:
    print(f"   -> CÓ. Câu dài nhất là {max_len} tokens.")
    print(f"   -> Số lượng câu > 100 tokens: {long_sentences} câu.")
    print("   -> Tác động: Với LSTM thuần (Vanilla LSTM), chuỗi > 100 tokens dễ gây ra 'Vanishing Gradient' (mất mát thông tin đầu câu).")
    print("   -> Giải pháp: Dùng Bi-LSTM (2 chiều) + Attention Mechanism hoặc cắt ngắn (Truncate) về mốc 99%.")
else:
    print("   -> KHÔNG. Dữ liệu này có độ dài vừa phải, LSTM xử lý tốt.")